# setup

In [18]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from efficientnet_pytorch import EfficientNet
from PIL import Image
from torchvision import transforms

from transformers import AutoModel, AutoTokenizer
import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader
import torch, gc

from sklearn.metrics.pairwise import cosine_similarity


HOME = '/data/git/shopee-product-matching'
pdata = f'{HOME}/data/shopee-product-matching'
!ls $pdata

BS = 256
NWKRS = 8
DEVICE = 'cuda'
PIN_MEMORY = True
MAXLEN = 128

sample_submission.csv  test.csv  test_images  train.csv  train_images


# data

In [2]:
fnm = 'train'
df = pd.read_csv(f'{pdata}/{fnm}.csv')
assert len(df) == df.posting_id.nunique()
p_imgs = f"{pdata}/train_images"
df.head(2)

,posting_id,image,image_phash,title,label_group
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045


# vision

In [3]:
class VDataset(Dataset):
    def __init__(self, df, p_imgs, transforms):
        self.df = df
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        img_id = self.df.image[idx]
        img_path = f"{p_imgs}/{img_id}"
        img = Image.open(img_path)
        img = self.transforms(img)
        return img

# https://github.com/lukemelas/EfficientNet-PyTorch
tfms = transforms.Compose([
    transforms.Resize((224,224)), 
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])

ds = VDataset(df, p_imgs, tfms)

dl = DataLoader(dataset=ds, 
                batch_size=BS,
                num_workers=NWKRS,
                pin_memory=PIN_MEMORY,
                shuffle=False)


device = torch.device(DEVICE)
mdlv = EfficientNet.from_pretrained('efficientnet-b0').to(device)
mdlv.eval()

feats = np.zeros((len(ds), 1280))
i = 0
for dat in tqdm(dl):
    with torch.no_grad():
        fts = mdlv.extract_features(dat.to(device)).mean(dim=(-1,-2))
    l = len(fts)
    feats[i:i+l,:] = fts.cpu().detach().numpy()

Loaded pretrained weights for efficientnet-b0


100%|██████████| 134/134 [01:55<00:00,  1.16it/s]


In [4]:
df.shape, feats.shape

((34250, 5), (34250, 1280))

In [5]:
del dl, ds, dat, fts, mdlv

In [6]:
gc.collect()
torch.cuda.empty_cache()

In [7]:
vfeats = feats

# text

In [8]:
def mk_tensors(txt, tokenizer, maxlen):
    tok_res = tokenizer(
        txt, truncation=True, padding=True, max_length=maxlen
    )
    input_ids = tok_res["input_ids"]
    token_type_ids = tok_res["token_type_ids"]
    attention_mask = tok_res["attention_mask"]
    input_ids = torch.tensor(input_ids)
    token_type_ids = torch.tensor(token_type_ids)
    attention_mask = torch.tensor(attention_mask)
    return input_ids, attention_mask, token_type_ids


def mk_dl(tensors):
    input_ids, token_type_ids, attention_mask = tensors
    ds = TensorDataset(input_ids, attention_mask, token_type_ids)
    dl = DataLoader(dataset=ds, 
                batch_size=BS,
                num_workers=NWKRS,
                pin_memory=PIN_MEMORY,
                shuffle=False)
    return dl

In [9]:
# 'bert-base-multilingual-cased',
# 'xlm-roberta-base',
# 'pvl/labse_bert'
nm_mdl = 'pvl/labse_bert'
tokenizer = AutoTokenizer.from_pretrained(nm_mdl, do_lower_case=False)

tensors = mk_tensors(list(df.title.values), tokenizer, maxlen=MAXLEN)

dl = mk_dl(tensors)

mdlt = AutoModel.from_pretrained(nm_mdl).to(device)
mdlt.eval()

feats = np.zeros((len(df), 768))
i = 0
for dat in tqdm(dl):
    with torch.no_grad():
        dat = (o.to(device) for o in dat)
        output = mdlt(*dat)
        fts = output.last_hidden_state
        fts = fts.mean(dim=-2)
        l = len(fts)
        feats[i:i+l,:] = fts.cpu().detach().numpy()

100%|██████████| 134/134 [01:13<00:00,  1.83it/s]


In [10]:
del dl, dat, fts, mdlt

In [11]:
gc.collect()
torch.cuda.empty_cache()

In [12]:
df.shape, feats.shape

((34250, 5), (34250, 768))

In [13]:
tfeats = feats